In [1]:
from src.OPFInterface import OPFClassifier
from functools import cmp_to_key
import pandas as pd
import numpy as np
from os import listdir
from src.utils import error, read_df
from tqdm import tqdm
# plot
import matplotlib.pyplot as plt
from sklearn.manifold import MDS

/mnt/c/Users/AndreFakhoury/Documents/GitHub/time-series-opf/venv/lib/python3.8/site-packages/numpy/core/getlimits.py:500: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/mnt/c/Users/AndreFakhoury/Documents/GitHub/time-series-opf/venv/lib/python3.8/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
/mnt/c/Users/AndreFakhoury/Documents/GitHub/time-series-opf/venv/lib/python3.8/site-packages/numpy/core/getlimits.py:500: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/mnt/c/Users/AndreFakhoury/Documents/GitHub/time-series-opf/venv/lib/python3.8/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type 

In [2]:
def analyse(df_name):
    X, y, X_test, y_test, df_errors = read_df(df_name)
    opf = OPFClassifier('euclidean-distance')
    opf.fit(X, y)
    preds = opf.classify(X_test)
    return error(preds, y_test), df_errors["ED"]

In [3]:
def get_val(a):
    datasets_df = pd.read_csv('data/DataSummary.csv')
    df_a = datasets_df.loc[datasets_df['Name'] == a].iloc[:,[3,6]]
    val_a1 = df_a.values[0][0] if len(df_a.values) else int(1e9)
    val_a2 = int(df_a.values[0][1]) if len(df_a.values) and df_a.values[0][1].isnumeric() else int(1e9)
    return val_a1 * val_a2

df_names = listdir('data/UCRArchive_2018')
df_values = [(get_val(df_name), df_name) for df_name in df_names]

df_values.sort()
df_names = [df_name for val, df_name in df_values if val < 1e6]

In [4]:
df_best = []
for df_name in tqdm(df_names):
    opf_error, nn_error = analyse(df_name)
    if opf_error < nn_error:
        df_best.append((df_name, opf_error, nn_error))

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 111/111 [00:53<00:00,  2.08it/s]


In [5]:
df_best.sort(key=lambda x: x[1]-x[2])
len(df_best)

42

In [ ]:
fig, axes = plt.subplots((len(df_best)+2)//3, 3, figsize=(25, 60))

def scatter_points(df_name, opf_error, nn_error, ax):
    X, y, X_test, y_test, df_errors = read_df(df_name)
    embedding = MDS(n_components=2)
    X_t = embedding.fit_transform(X)

    for cur_y in np.unique(y):
        ax.scatter(X_t[y==cur_y,0], X_t[y==cur_y,1])
    ax.legend(np.unique(y))
    ax.title.set_text("%s - OPF %.3f vs NN %.3f" % (df_name, opf_error, nn_error))
    
for idx, df_info in enumerate(df_best):
    df_name, opf_error, nn_error = df_info
    print(df_name)
    scatter_points(df_name, opf_error, nn_error, axes.flatten()[idx])

plt.savefig('scatter.pdf')
plt.show()

SemgHandSubjectCh2
InsectEPGSmallTrain
InsectEPGRegularTrain
SemgHandMovementCh2
PigArtPressure
SemgHandGenderCh2
MelbournePedestrian
GunPointAgeSpan
PigCVP
GunPointOldVersusYoung
PowerCons
SmoothSubspace
DodgerLoopDay
PigAirwayPressure
UMD
GunPointMaleVersusFemale
HouseTwenty
Fungi
Herring
ToeSegmentation1
Crop
